# Proyecto 1: Cuanti

## Clase

In [147]:
import math

class Cuanti:

    def __init__(self, nombre='X', data=[]):
        self.nombre = nombre
        self.data = data

    def set_nombre(self, nuevo_nombre):
        self.nombre = nuevo_nombre

    def get_nombre(self):
        return self.nombre

    def set_data(self, nueva_data):
        self.data = nueva_data
    
    def media(self):
        return round(sum(self.data) / len(self.data), 2)

    def media2(self):
        cont = 0
        acum = 0
        for elem in self.data:
            cont += 1
            acum += elem
        return round(acum / cont, 2)
    
    def minimo(self):
        return min(self.data)
    
    def mediana(self):
        n = len(self.data)
        centro = int(n/2)
        ordenada = sorted(self.data)
        if n%2 != 0:
            return ordenada[centro]
        return Cuanti(data=[ordenada[centro], ordenada[centro - 1]]).media()
    
    def maximo(self):
        return max(self.data)

    def rango(self):
        return self.maximo() - self.minimo()

    def coeficiente_variacion(self):
        return round(self.desvio_estandar() / self.media(), 2)

    def moda(self):
        dicc = {}
        for elem in self.data:
            dicc[elem] = (1 if elem not in dicc else (dicc[elem] + 1))
        maximo = max(dicc.values())
        moda = []
        for elem in dicc.keys():
            if dicc[elem] == maximo:
                moda.append(elem)
        return sorted(moda)
            
    def varianza(self):
        acum = 0
        media = self.media()
        for elem in self.data:
            acum += pow(elem - media, 2)
        return round(acum / (len(self.data) - 1), 2)
        
    def desvio_estandar(self):
        return round(math.sqrt(self.varianza()), 2)
    
    def q1(self):
        n = len(self.data)
        centro = int(n/2)
        ordenada = sorted(self.data)   
        return Cuanti(data=ordenada[:centro]).mediana()
    
    def q3(self):
        n = len(self.data)
        centro = int(n/2)
        ordenada = sorted(self.data)
        if n%2 != 0:
            return Cuanti(data=ordenada[centro+1:]).mediana()
        return Cuanti(data=ordenada[centro:]).mediana()

    def iqr(self):
        return self.q3() - self.q1()
        
    def atipicos(self):
        umbral_inf = 1.5 * self.iqr()
        umbral_sup = 3 * self.iqr()
        atipicos = []
        for elem in self.data:
            if elem >= umbral_inf and elem < umbral_sup:
                atipicos.append(elem)
        return sorted(atipicos)
    
    def muy_atipicos(self):
        umbral_inf = 3 * self.iqr()
        muy_atipicos = []
        for elem in self.data:
            if elem >= umbral_inf:
                muy_atipicos.append(elem)
        return sorted(muy_atipicos)
        
    def medidas_resumen(self):
        print('Nombre: ',  self.nombre)
        print('Datos: ',   self.data)
        print('Media: ',   self.media())
        print('Mediana: ', self.mediana())

    def __str__(self):
        return self.nombre + '\n' + str(self.data)

In [148]:
c = Cuanti('v1', [1, 2, 3])

In [149]:
c.get_nombre()

'v1'

In [150]:
c.set_nombre('cantidades')

In [151]:
c.get_nombre()

'cantidades'

In [152]:
c.media()

2.0

In [153]:
c.minimo()

1

In [154]:
c.maximo()

3

In [155]:
c.rango()

2

In [156]:
c.mediana()

2

In [157]:
print(c)

cantidades
[1, 2, 3]


In [158]:
c.set_nombre('v')

In [159]:
print(c)

v
[1, 2, 3]


In [160]:
c.medidas_resumen()

Nombre:  v
Datos:  [1, 2, 3]
Media:  2.0
Mediana:  2


## Tests

In [161]:
import unittest
import sqlite3

class TestNombreCuanti(unittest.TestCase):

    def setUp(self):
        # hardcode
        self.c1 = Cuanti('v1', [1, 3, 4])
        self.c2 = Cuanti('v2', [100, 200, 200, 250, 600])

        # file
        fr = open("/Users/ppando/Materias/python/proy/cuanti_datos.csv", 'r')
        lista2 = fr.readlines()
        fr.close()
        self.datos = []
        for elem in lista2:                      
            self.datos.append(int(elem.replace('\n', '')))
        self.c3 = Cuanti('v3', self.datos)

        # database
        self.con = sqlite3.connect("/Users/ppando/Materias/python/proy/cuanti_datos.db") # si no existe, la crea
        cur = self.con.cursor()
        cur.execute("CREATE TABLE datos(id INTEGER PRIMARY KEY AUTOINCREMENT, value INTEGER NOT NULL)")
        cur.execute("INSERT INTO datos (value) VALUES (100), (200), (200), (250), (600)")
        self.con.commit()
        self.datos = []
        for row in cur.execute("SELECT value FROM datos"):                      
            self.datos.append(row[0])
        self.c4 = Cuanti('v4', self.datos)
        
    def test_nombre(self):
        self.assertEqual(self.c1.get_nombre(), 'v1')
        self.assertEqual(self.c2.get_nombre(), 'v2')

    def test_nombre_nuevo(self):
        self.c1.set_nombre('cantidades')
        self.assertEqual(self.c1.get_nombre(), 'cantidades')

    def test_media(self):
        self.assertEqual(self.c1.media(), 2.67)
        self.assertEqual(self.c2.media(), 270)
        self.assertEqual(self.c3.media(), 270)
        self.assertEqual(self.c4.media(), 270)
                
    def test_media2(self):
        self.assertEqual(self.c1.media2(), 2.67)
        self.assertEqual(self.c2.media2(), 270)
        self.assertEqual(self.c3.media2(), 270)
        self.assertEqual(self.c4.media2(), 270)

    def test_minimo(self):
        self.assertEqual(self.c1.minimo(), 1)
        self.assertEqual(self.c2.minimo(), 100)
        self.assertEqual(self.c3.minimo(), 100)
        self.assertEqual(self.c4.minimo(), 100)
        
    def test_maximo(self):
        self.assertEqual(self.c1.maximo(), 4)
        self.assertEqual(self.c2.maximo(), 600)
        self.assertEqual(self.c3.maximo(), 600)
        self.assertEqual(self.c4.maximo(), 600)

    def test_rango(self):
        self.assertEqual(self.c1.rango(), 3)
        self.assertEqual(self.c2.rango(), 500)
        self.assertEqual(self.c3.rango(), 500)
        self.assertEqual(self.c4.rango(), 500)
        
    def test_mediana_n_impar(self):
        self.assertEqual(self.c1.mediana(), 3)
        self.assertEqual(self.c2.mediana(), 200)
        self.assertEqual(self.c3.mediana(), 200)
        
    def test_mediana_n_par(self): 
        self.assertEqual(Cuanti('v2', [9, 5, 6, 1]).mediana(), 5.5)
        self.assertEqual(Cuanti('v', [100, 200, 900, 250, 600, 350]).mediana(), 300)

    def test_moda(self):
        self.assertEqual(self.c1.moda(), [1, 3, 4])
        self.assertEqual(self.c2.moda(), [200])
        self.assertEqual(self.c3.moda(), [200])
        self.assertEqual(self.c4.moda(), [200])

    def test_varianza(self):
        self.assertEqual(self.c1.varianza(), 2.33)
        self.assertEqual(self.c2.varianza(), 37000)
        self.assertEqual(self.c3.varianza(), 37000)
        self.assertEqual(self.c4.varianza(), 37000)

    def test_desvio_estandar(self):
        self.assertEqual(self.c1.desvio_estandar(), 1.53)
        self.assertEqual(self.c2.desvio_estandar(), 192.35)
        self.assertEqual(self.c3.desvio_estandar(), 192.35)
        self.assertEqual(self.c4.desvio_estandar(), 192.35)

    def test_q1(self):
        self.assertEqual(Cuanti(data=[1, 5, 6, 9]).q1(), 3)
        self.assertEqual(Cuanti(data=[13, 13.5, 14, 14, 14.5, 15.5, 16]).q1(), 13.5)
        self.assertEqual(self.c1.q1(), 1)
        self.assertEqual(self.c2.q1(), 150)
        
    def test_q3(self):
        self.assertEqual(Cuanti(data=[1, 5, 6, 9]).q3(), 7.50)
        self.assertEqual(Cuanti(data=[13, 13.5, 14, 14, 14.5, 15.5, 16]).q3(), 15.5)
        self.assertEqual(self.c1.q3(), 4)
        self.assertEqual(self.c2.q3(), 425)
        
    def test_iqr(self):
        self.assertEqual(Cuanti(data=[1, 5, 6, 9]).iqr(), 4.50)
        self.assertEqual(Cuanti(data=[13, 13.5, 14, 14, 14.5, 15.5, 16]).iqr(), 2)
        self.assertEqual(self.c1.iqr(), 3)
        self.assertEqual(self.c2.iqr(), 275)
        
    def test_atipicos(self):
        self.assertEqual(Cuanti('v', [100, 200, 900, 250, 600, 350, 3000]).atipicos(), [])
        self.assertEqual(self.c1.atipicos(), [])
        self.assertEqual(self.c2.atipicos(), [600])
        
    def test_muy_atipicos(self):
        self.assertEqual(Cuanti('v', [100, 200, 900, 250, 600, 350, 3000]).muy_atipicos(), [3000])
        self.assertEqual(self.c1.muy_atipicos(), [])
        self.assertEqual(self.c2.muy_atipicos(), [])

    def tearDown(self):
        self.con.cursor().execute("DROP TABLE datos")
        self.con.close()

unittest.main(argv=[''], verbosity=2, exit=False)

test_media (__main__.TestCuantiHardCode.test_media) ... ok
test_mediana_n_impar (__main__.TestCuantiHardCode.test_mediana_n_impar) ... ok
test_mediana_n_par (__main__.TestCuantiHardCode.test_mediana_n_par) ... ok
test_rango (__main__.TestCuantiHardCode.test_rango) ... ok
test_varianza (__main__.TestCuantiHardCode.test_varianza) ... ok
test_atipicos (__main__.TestNombreCuanti.test_atipicos) ... ok
test_desvio_estandar (__main__.TestNombreCuanti.test_desvio_estandar) ... ok
test_iqr (__main__.TestNombreCuanti.test_iqr) ... ok
test_maximo (__main__.TestNombreCuanti.test_maximo) ... ok
test_media (__main__.TestNombreCuanti.test_media) ... ok
test_media2 (__main__.TestNombreCuanti.test_media2) ... ok
test_mediana_n_impar (__main__.TestNombreCuanti.test_mediana_n_impar) ... ok
test_mediana_n_par (__main__.TestNombreCuanti.test_mediana_n_par) ... ok
test_minimo (__main__.TestNombreCuanti.test_minimo) ... ok
test_moda (__main__.TestNombreCuanti.test_moda) ... ok
test_muy_atipicos (__main__.Tes